# 8760 Load Analysis 
## Purpose
The goal of this code is to take 8760 .csv files from electrical system and analyze when and where the load is not being met. It was specifically designed with the output from the HOMER microgrid software in mind. 

In [4]:
# import modules 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import os 
import pathlib
from glob  import glob

In [5]:
# define functions

def count_consecutive_groups(df):
    groups = (df['hour'].diff() != 1).cumsum()  # Identify consecutive groups
    return len(groups.unique()) 


In [ ]:
#load csv file file path 
dir_path = os.path.join(
    pathlib.Path.home(),
        'code-projects',
        'load-8760'
        )


data_files = glob(os.path.join(dir_path,'*.csv'))

#list to store all results

results = []
outages_day_hrs_comb=[]
outages_per_day_comb=[]

for data_path in data_files:
    data = pd.read_csv(
        data_path,
        delimiter=',',
        header= 0,
        index_col='Time',
        skiprows=1)
     
    data= data[data.index.notnull()]

    data=data.reset_index() #Reset Index
    data["Time"] = pd.to_datetime(data["Time"])
    data["hour"] = data['Time'].map(lambda x: x.hour)
    data["day"] = data['Time'].map(lambda x: x.day)
    data["month"] = data['Time'].map(lambda x: x.month)
    

     #Initialize N/A
    hours_shortage = 'NA'
    hours_battery_shortage = 'NA'
    hours_unmet_load = 'NA' 
    num_days_outages = 'NA' 
    outages_year = 'NA' 
    # filter  how many hours load not met (capacity shortage > 0)
    if 'Capacity Shortage' in data.columns:
        data['Capacity Shortage'] = data['Capacity Shortage'].astype(float) #first need to force column to convert to float 
        capacity_shortage_df = data[data['Capacity Shortage'] > 0 ]
        hours_shortage = len(capacity_shortage_df)
   

    #Calculate how many hours annual load is unmet   #for Battery <30%
    if 'Generic 1kWh Li-Ion State of Charge' in data.columns:
        data['Generic 1kWh Li-Ion State of Charge'] = data['Generic 1kWh Li-Ion State of Charge'].astype(float)
        battery_shortage_df = data[data['Generic 1kWh Li-Ion State of Charge'] < 30 ]
        hours_battery_shortage = len(battery_shortage_df)
    # To find hours with Unmet Electrical Load

    if 'Unmet Electrical Load' in data.columns:
        data['Unmet Electrical Load'] = data['Unmet Electrical Load'].astype(float)
        unmet_load_df = data[data['Unmet Electrical Load'] > 0.01 ]
        hours_unmet_load = len(unmet_load_df)
    
    hours_per_day = unmet_load_df.groupby(['month','day']).size()
   )
    #Calculate number of days with outages 
    num_days_outages = len(hours_per_day)
    # Calulate number of outages per year
    outages_year = count_consecutive_groups (unmet_load_df)
    #Calculate number of distinct outages 
    outages_day = unmet_load_df.groupby(['month', 'day']).apply(count_consecutive_groups)
    #complile a list of results 
    results.append ({
         'File': os.path.basename(data_path),
        'Hours with Capacity Shortage': hours_shortage,
        'Hours with Battery <30%': hours_battery_shortage,
        'Hours with Unmet Load': hours_unmet_load,
        'Days with Outages': num_days_outages,
        'Total Outages (Year)': outages_year
    })
results_df = pd.DataFrame(results)

results_df

C:\Users\Nolan Welsh\AppData\Local\Temp\ipykernel_26400\1189129509.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data["Time"] = pd.to_datetime(data["Time"])
C:\Users\Nolan Welsh\AppData\Local\Temp\ipykernel_26400\1189129509.py:63: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  outages_day = unmet_load_df.groupby(['month', 'day']).apply(count_consecutive_groups)
C:\Users\Nolan Welsh\AppData\Local\Temp\ipykernel_26400\1189129509.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure p

,File,Hours with Capacity Shortage,Hours with Battery <30%,Hours with Unmet Load,Days with Outages,Total Outages (Year)
0,0pctCapShortage.csv,NA,104,29,7,7
1,2-5pctCapShortage.csv,432,551,349,49,62
2,5pctCapShort.csv,721,789,592,72,101
3,7-5pctCapShortage.csv,987,1112,815,97,133


Save Summary  results 

In [ ]:

summary_path = os.path.join(dir_path, 'shortage_summary_results.csv')
results_df.to_csv(summary_path, index=False)



month  day
1      1      10
       2       7
       4       7
       5      14
       6       7
       13      3
2      3       4
       8      12
       9       9
       10      7
3      24      2
       27     12
       28      6
4      8      10
       9       6
       11      3
       29      1
       30      6
5      8      13
       9       6
       13      9
       14      6
       17     11
       18     15
       19      6
6      17     14
       18     13
       19      6
       25      4
       26     18
       27      6
7      21      1
       22      6
8      2      19
       3      20
       4       6
9      2       8
       3      13
       4       6
       15     10
       16      6
       17      6
       27     13
       28      6
       29      8
       30      6
10     11     11
       12      6
       18      1
       29      4
       30      7
       31     10
11     1       6
       8      13
       9      14
       10      6
       18      9
       19      6
   